In [309]:
from attributes import *
import polars as pl
import altair as alt

# get data
df = pl.read_csv("data/2_research_ethic_recognition.csv")

In [310]:
def basic_analysis(name):
    count = df.group_by(name).len().sort(name)
    proportion = count.with_columns(
        pct = pl.col("len") / pl.col("len").sum() * 100
    )
    mean = df.select(mean = pl.col(name).mean())
    std = df.select(std = pl.col(name).std())
    

In [311]:
def draw_bar_plot(df, x, y, title, MAP, rotate=True, legend=True):

    bar_x = alt.X(x, title=title, axis=None)
    bar_x = bar_x if MAP is None else bar_x.sort(MAP.values())

    txt_x = alt.X(x, sort=MAP.values()) if MAP is not None else alt.X(x)
    lgd_x = alt.Legend(orient='bottom', direction='horizontal')
    color = alt.Color(x, title=None, legend=lgd_x) if legend else alt.Color(x, title=None, legend=None)
    color = color.sort(MAP.values()) if MAP is not None else color

    bar = df.plot.bar().encode(
        x=bar_x,
        y=alt.Y(y, title=None),
        color=color,
    ).properties(
        title=title,
        width=400,
    )

    text = bar.mark_text(
        align="center",
        baseline="bottom",
    ).encode(
        x=txt_x,
        y=y,
        text=y,
        color=alt.value("black"),
    )
    return bar + text

In [312]:
def get(title, __2022, __2023, importance):

    __2022 = __2022.with_columns(
        pct2022 = pl.col(title) / pl.col(title).sum() * 100,
    )

    __2023 = __2023.with_columns(
        pct2023 = pl.col(title) / pl.col(title).sum() * 100,
    )

    _2024 = df.group_by(title).len().sort(title)

    if _2024[title].len() < len(importance):
        extending = pl.DataFrame({
            title: [1],
            "len": [0]
        }).cast(pl.Schema({ "len": pl.UInt32()}))
        _2024 = _2024.extend(extending)

    _2024 = _2024.rename({"len": "2024"}).sort(title)
    _2024 = _2024.with_columns(
        pct2024 = pl.col("2024") / pl.col("2024").sum() * 100,
        mean2024 = (pl.col(title) * pl.col("2024")).sum() / _2024["2024"].sum(),
    )
    _2024 = _2024.with_columns(
        std2024 = (((pl.col(title) - _2024["mean2024"]) ** 2 * pl.col("2024")).sum() / pl.col("2024").sum()).sqrt(),
    )

    all = __2022.join(__2023, on=title, how="left").join(_2024, on=title, how="left")
    all = all.with_columns(
        total = pl.col("2022") + pl.col("2023") + pl.col("2024")
    )

    all = all.with_columns(
        pctAll = all["total"] / all["total"].sum() * 100,
        meanAll = (pl.col(title) * pl.col("total")).sum() / all["total"].sum(),
    )
    all = all.with_columns(
        stdAll = (((pl.col(title) - all["meanAll"]) ** 2 * pl.col("total")).sum() / pl.col("total").sum()).sqrt(),
    )
    return all

In [313]:
def draw_graph(_data, title, question):
    data_long = _data.select(title, "2022", "2023", "2024").melt(id_vars=[title], variable_name="Year", value_name="Count")

    stacked_bar = alt.Chart(data_long).mark_bar(size=100).encode(
        x=alt.X("Year:O", title=None, axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("Count:Q", title=None).stack('normalize'),
        color=alt.Color(f"{title}:N", title=title),
        tooltip=[title, "Year", "Count"]
    ).properties(
        width=600,
        height=400
    )
    stacked_bar.save(f"figure/7/{question}.png")
    stacked_bar.show()

In [314]:
# 
# 가. 연구자의 연구윤리 인식과 실천의 중요성 인식 
# 
title = "연구윤리 인식 및 실천의 중요성 인식"

importance = [1, 2, 3, 4, 5]

# 1) counting
_2022 = pl.DataFrame({
    "Q1": importance,
    "2022": [9, 0, 4, 47, 103],
})

_2023 = pl.DataFrame({
    "Q1": importance,
    "2023": [0, 0, 2, 56, 112],
})

# basic_analysis("Q1")
data = get("Q1", _2022, _2023, importance)
draw_graph(data, "Q1", "연구자의 연구윤리 인식과 실천의 중요성 인식")

C:\Users\hyeon\AppData\Local\Temp\ipykernel_21808\1464162698.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select(title, "2022", "2023", "2024").melt(id_vars=[title], variable_name="Year", value_name="Count")


alt.Chart(...)

In [315]:
# from scipy import stats
# # 
# # 나. 연구자의 연구윤리 준수 수준 
# # 

# # # 1) counting
# # _2022 = pl.DataFrame({
# #     "Q2": importance,
# #     "2022": [1, 5, 42, 89, 26],
# # })

# # _2023 = pl.DataFrame({
# #     "Q2": importance,
# #     "2023": [0, 6, 55, 75, 34],
# # })

# # get("Q2", _2022, _2023)

In [316]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("연구윤리교육")
importance = [1, 2, 3, 4, 5]

title ="Q3_1"

# 1) counting
_2022 = pl.DataFrame({
    title: importance,
    "2022": [1, 2, 33, 88, 39],
})

_2023 = pl.DataFrame({
    title: importance,
    "2023": [2, 4, 38, 90, 36],
})

data = get(title, _2022, _2023, importance)
draw_graph(data, title, "연구윤리교육")

C:\Users\hyeon\AppData\Local\Temp\ipykernel_21808\1464162698.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select(title, "2022", "2023", "2024").melt(id_vars=[title], variable_name="Year", value_name="Count")


연구윤리교육


alt.Chart(...)

In [317]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("연구윤리 관련 보도")
importance = [1, 2, 3, 4, 5]

title ="Q3_2"

# 1) counting
_2022 = pl.DataFrame({
    title: importance,
    "2022": [1, 5, 42, 89, 26],
})

_2023 = pl.DataFrame({
    title: importance,
    "2023": [0, 6, 55, 75, 34],
})

data = get(title, _2022, _2023, importance)
draw_graph(data, title, "연구윤리 관련 보도")

연구윤리 관련 보도


C:\Users\hyeon\AppData\Local\Temp\ipykernel_21808\1464162698.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select(title, "2022", "2023", "2024").melt(id_vars=[title], variable_name="Year", value_name="Count")


alt.Chart(...)

In [318]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("연구자 간 멘토링")
importance = [1, 2, 3, 4, 5]

title ="Q3_3"

# 1) counting
_2022 = pl.DataFrame({
    title: importance,
    "2022": [1, 1, 13, 81, 67],
})

_2023 = pl.DataFrame({
    title: importance,
    "2023": [0, 3, 18, 8, 65],
})

_2024 = df.group_by(title).len().sort(title)

data = get(title, _2022, _2023, importance)
draw_graph(data, title, "연구자 간 멘토링")

연구자 간 멘토링


C:\Users\hyeon\AppData\Local\Temp\ipykernel_21808\1464162698.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select(title, "2022", "2023", "2024").melt(id_vars=[title], variable_name="Year", value_name="Count")


alt.Chart(...)

In [319]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("관련 규정 및 지침")

title ="Q3_4"

# 1) counting
_2022 = pl.DataFrame({
    title: importance,
    "2022": [1, 2, 27, 90, 43],
})

_2023 = pl.DataFrame({
    title: importance,
    "2023": [0, 1, 24, 86, 59],
})

_2024 = df.group_by(title).len().sort(title)

data = get(title, _2022, _2023, importance)
draw_graph(data, title, "관련 규정 및 지침")

관련 규정 및 지침


C:\Users\hyeon\AppData\Local\Temp\ipykernel_21808\1464162698.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select(title, "2022", "2023", "2024").melt(id_vars=[title], variable_name="Year", value_name="Count")


alt.Chart(...)

In [320]:
# 
# 다. 연구자의 연구윤리 준수 영향 요인별
# 
print("부정행위 검증 및 제재")

title ="Q3_5"

# 1) counting
_2022 = pl.DataFrame({
    title: importance,
    "2022": [1, 1, 28, 71, 62],
})

_2023 = pl.DataFrame({
    title: importance,
    "2023": [0, 2, 24, 77, 67],
})

_2024 = df.group_by(title).len().sort(title)

data = get(title, _2022, _2023, importance)
draw_graph(data, title, "부정행위 검증 및 제재")

부정행위 검증 및 제재


C:\Users\hyeon\AppData\Local\Temp\ipykernel_21808\1464162698.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select(title, "2022", "2023", "2024").melt(id_vars=[title], variable_name="Year", value_name="Count")


alt.Chart(...)

In [321]:
# 
# 라. KIOST에 대한 인식
# 
print("KIOST의 연구부정행위 제보 방법 인식 여부")

df = pl.read_csv("data/4_KIOST_research_ethics_system.csv")

title ="Q26"
importance = [1, 2]

# 1) counting
_2022 = pl.DataFrame({
    title: importance,
    "2022": [74, 89],
})

_2023 = pl.DataFrame({
    title: importance,
    "2023": [80, 90],
})

_2024 = df.group_by(title).len().sort(title)

data = get(title, _2022, _2023, importance)
draw_graph(data, title, "KIOST의 연구부정행위 제보 방법 인식 여부")

KIOST의 연구부정행위 제보 방법 인식 여부


C:\Users\hyeon\AppData\Local\Temp\ipykernel_21808\1464162698.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select(title, "2022", "2023", "2024").melt(id_vars=[title], variable_name="Year", value_name="Count")


alt.Chart(...)

In [322]:
# 
# 라. KIOST에 대한 인식
# 
# 2) KIOST의 연구부정행위 제보 접수 및 처리 절차 인지
print("KIOST의 연구부정행위 제보 접수 및 처리의 적정성")

title ="Q28"
importance = [1, 2, 3, 4, 5]

# 1) counting
_2022 = pl.DataFrame({
    title: importance,
    "2022": [4, 14, 88, 44, 13],
})

_2023 = pl.DataFrame({
    title: importance,
    "2023": [7, 19, 78, 51, 15],
})

_2024 = df.group_by(title).len().sort(title)

data = get(title, _2022, _2023, importance)
draw_graph(data, title, "KIOST의 연구부정행위 제보 접수 및 처리의 적정성")


KIOST의 연구부정행위 제보 접수 및 처리의 적정성


C:\Users\hyeon\AppData\Local\Temp\ipykernel_21808\1464162698.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select(title, "2022", "2023", "2024").melt(id_vars=[title], variable_name="Year", value_name="Count")


alt.Chart(...)

In [323]:
# 
# 라. KIOST에 대한 인식
# 
# 3) KIOST의 연구부정행위 검증이 공정한가에 대한 평가
print("KIOST의 연구부정행위 검증이 공정한가에 대한 평가")

title ="Q29"
importance = [1, 2, 3, 4, 5]

# 1) counting
_2022 = pl.DataFrame({
    title: importance,
    "2022": [4, 14, 84, 47, 14],
})

_2023 = pl.DataFrame({
    title: importance,
    "2023": [1, 14, 74, 68, 13],
})

_2024 = df.group_by(title).len().sort(title)

data = get(title, _2022, _2023, importance)
draw_graph(data, title, "KIOST의 연구부정행위 검증이 공정한가에 대한 평가")

KIOST의 연구부정행위 검증이 공정한가에 대한 평가


C:\Users\hyeon\AppData\Local\Temp\ipykernel_21808\1464162698.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  data_long = _data.select(title, "2022", "2023", "2024").melt(id_vars=[title], variable_name="Year", value_name="Count")


alt.Chart(...)

In [324]:
# 
# 마. KIOST 연구자의 연구윤리 지수 수준
# 

research_ethics_score = pl.read_csv("data/5_1_research_ethics_score.csv")

# *Q39_1: 동물실험에 대한 연구윤리
#
공정 = (pl.col("Q38_1") + pl.col("Q38_2") + pl.col("Q38_3") + pl.col("Q38_4")) / 4
정직 = (pl.col("Q38_5") + pl.col("Q38_6") + pl.col("Q38_7") + pl.col("Q38_8")) / 4
존중 = pl.when(pl.col("Q39_1").is_null()).then(
    (pl.col("Q38_9") + pl.col("Q38_10") + pl.col("Q38_11")) / 3
).otherwise(
    (pl.col("Q38_9") + pl.col("Q38_10") + pl.col("Q38_11") + pl.col("Q39_1")) / 4
)

책임 = (pl.col("Q38_12") + pl.col("Q38_13") + pl.col("Q38_14") + pl.col("Q38_15")) / 4
투명성 = (pl.col("Q38_16") + pl.col("Q38_17") + pl.col("Q38_18") + pl.col("Q38_19")) / 4
all = (공정 + 정직 + 존중 + 책임 + 투명성) / 5

by_area = research_ethics_score.select([
    pl.col("ID"),
    공정.alias("공정"),
    정직.alias("정직"),
    존중.alias("존중"),
    책임.alias("책임"),
    투명성.alias("투명성"),
    all.alias("전체")
])


def make_grade(column, excellent, great, good, bad):
    return pl.when(column >= excellent).then(5).when(column >= great).then(4).when(column >= good).then(3).when(column >= bad).then(2).otherwise(1)

grading_all = make_grade(pl.col("전체"), 4.70, 3.85, 3.40, 2.95)
grading_공정 = make_grade(pl.col("공정"), 5.0, 3.75, 3.0, 2.50)
grading_정직 = make_grade(pl.col("정직"), 5.0, 4.0, 3.5, 3.25)
grading_존중 = make_grade(pl.col("존중"), 5.0, 3.75, 3.0, 2.25)
grading_책임 = make_grade(pl.col("책임"), 5.0, 3.75, 3.25, 2.75)
grading_투명성 = make_grade(pl.col("투명성"), 5.0, 3.75, 3.25, 2.75)

grading = by_area.select([
    grading_all.alias("전체_채점"),
    grading_공정.alias("공정_채점"),
    grading_정직.alias("정직_채점"),
    grading_존중.alias("존중_채점"),
    grading_책임.alias("책임_채점"),
    grading_투명성.alias("투명성_채점"),
])

all_count = grading.group_by("전체_채점").agg(pl.count("전체_채점").alias("전체")).rename({"전체_채점": "등급"})
공정_count = grading.group_by("공정_채점").agg(pl.count("공정_채점").alias("공정")).rename({"공정_채점": "등급"})
정직_count = grading.group_by("정직_채점").agg(pl.count("정직_채점").alias("정직")).rename({"정직_채점": "등급"})
존중_count = grading.group_by("존중_채점").agg(pl.count("존중_채점").alias("존중")).rename({"존중_채점": "등급"})
책임_count = grading.group_by("책임_채점").agg(pl.count("책임_채점").alias("책임")).rename({"책임_채점": "등급"})
투명성_count = grading.group_by("투명성_채점").agg(pl.count("투명성_채점").alias("투명성")).rename({"투명성_채점": "등급"})

def check_zero_and_concat(dataframe, column_name, basis=5):
    if dataframe.shape[0] < basis:
        t = pl.DataFrame({"count": [1], column_name: [0]})
        t = t.with_columns(
            pl.col("count").cast(pl.Int32),
            pl.col(column_name).cast(pl.UInt32),
        )
        return dataframe.extend(t)
    return dataframe

all_count = all_count.join(공정_count, on="등급", how="left") \
    .join(정직_count, on="등급", how="left") \
    .join(존중_count, on="등급", how="left") \
    .join(책임_count, on="등급", how="left") \
    .join(투명성_count, on="등급", how="left") \
    .fill_null(0) \
    .sort("등급")

all_count.drop("등급").transpose(
    include_header=True
).rename(
    {"column": "영역", "column_0":"1", "column_1":"2", "column_2":"3", "column_3":"4", "column_4":"5"}
).select(
    영역 = pl.col("영역"),
    평균 = (pl.col("1") + pl.col("2") * 2 + pl.col("3") * 3 + pl.col("4") * 4 + pl.col("5") * 5) / (pl.col("1") + pl.col("2") + pl.col("3") + pl.col("4") + pl.col("5")),
    미흡 = (pl.col("1") + pl.col("2")),
    보통 = pl.col("3"),
    우수 = (pl.col("4") + pl.col("5")),
    
).with_columns(
    미흡_pct = pl.col("미흡") / (pl.col("미흡") + pl.col("보통") + pl.col("우수")) * 100,
    보통_pct = pl.col("보통") / (pl.col("미흡") + pl.col("보통") + pl.col("우수")) * 100,
    우수_pct = pl.col("우수") / (pl.col("미흡") + pl.col("보통") + pl.col("우수")) * 100,
)

영역,평균,미흡,보통,우수,미흡_pct,보통_pct,우수_pct
str,f64,u32,u32,u32,f64,f64,f64
"""전체""",4.113772,11,12,144,6.586826,7.185629,86.227545
"""공정""",4.023952,1,23,143,0.598802,13.772455,85.628743
"""정직""",4.167665,12,9,146,7.185629,5.389222,87.42515
"""존중""",4.197605,1,18,148,0.598802,10.778443,88.622754
"""책임""",4.023952,10,16,141,5.988024,9.580838,84.431138
"""투명성""",4.113772,12,5,150,7.185629,2.994012,89.820359
